# Player Facts Preparation
## Collecting NHL Players' Facts from EliteProspects
This step gathers player facts data for downstream NHL prediction.

- Collect league-wide player stats for a given season from
https://eliteprospects.com/league/{league}/stats/{season}?page={n}

- Extract metadata, including individual player profile links.

- Build a unique player list from seasons 2000–2025 by storing profile links in a set.

- Scrape each player’s page to retrieve their first 5 years of NHL stats, filtered by seasons where they played more than 30 games.

- Gather pre-NHL stats for each player (junior, college, international, etc.).

## Import Libraries

In [35]:
import eliteprospects_scraper_api as ep
import pandas as pd
import os
import time
import random

### Collecting NHL Players' Metadata from EliteProspects

In [4]:
if not os.path.exists('./data/nhl/nhl_players_metadata.csv'):
    # Collect players from 2000-2025 season
    for i in range(0, 25):
        season = f'20{str(i).zfill(2)}-20{str(i + 1).zfill(2)}'
        print(f'Scraping {season}')
        nhl_players = ep.get_season_roster("nhl", season)
        nhl_players.to_csv(f'./data/nhl/players/nhl_players_{season}.csv', index=False, encoding='utf-8-sig')
        print(f'Finished scraping {season}')
else:
    print('NHL players metadata already exists. Skipping scraping.')

NHL players metadata already exists. Skipping scraping.


In [5]:
if not os.path.exists('./data/nhl/nhl_players_metadata.csv'):
    # Collect players' metadata from 2000-2025 season
    nhl_players_metadata = pd.DataFrame()
    for i in range(0, 25):
        season = f'20{str(i).zfill(2)}-20{str(i + 1).zfill(2)}'
        nhl_players = pd.read_csv(f'./data/nhl/players/nhl_players_{season}.csv')
        nhl_players_metadata = pd.concat([nhl_players_metadata, ep.get_players_metadata(nhl_players)]).reset_index(drop=True)
        # Remove duplicates
        nhl_players_metadata = nhl_players_metadata.drop_duplicates(subset=['player_name']).reset_index(drop=True)
        print(f'Finished Merging Metadata for {season}')
else:
    print('NHL players metadata already exists. Skipping merging.')
    nhl_players_metadata = pd.read_csv('./data/nhl/nhl_players_metadata.csv')


NHL players metadata already exists. Skipping merging.


### Testing API

In [2]:
# Read in nhl_skaters_metadata_official_ep_merge_complete_final.csv
nhl_players_metadata = pd.read_csv('./data/nhl/nhl_skaters_metadata_official_ep_merge_complete_final.csv')

In [3]:
from selenium.webdriver.support.wait import WebDriverWait
import undetected_chromedriver as uc

# Set up Chrome Driver ONCE
chrome_options = uc.ChromeOptions()
chrome_options.add_argument("--headless=new")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")

driver = uc.Chrome(version_main=138, options=chrome_options)
wait = WebDriverWait(driver, 15)

In [4]:
cale_makar_metadata = nhl_players_metadata.iloc[3188]
cale_makar_metadata

player_name                                           Cale Makar
fw_def                                                       DEF
link           https://www.eliteprospects.com/player/199655/c...
Name: 3188, dtype: object

In [5]:
cale_makar_facts = ep.get_player_facts_with_reusable_driver(cale_makar_metadata, driver, wait)

Extracting main facts...
Extracting extra facts...
Extracted extra fact: Player Type
Extracted extra fact: Cap Hit
Extracted extra fact: NHL Rights
Extracted extra fact: Drafted
Extracted extra fact: Agency
Extracted extra fact: Highlights
Extracting highlights...
Extracting player types...
Extracting description...


In [6]:
cale_makar_facts

,player_name,nation,position,height_cm,weight_kg,shoots,player_type,nhl_rights,draft,highlights,description
0,Cale Makar,Canada,D,180,85,R,"[PP Specialist, Speedster, Two-Way Defenseman]",Colorado Avalanche / Signed,"(1, 4, 2017)","[1-time WJAC-19 Gold Medal, 1-time U20 WJC Gol...",Many have called Cale Makar one of the purest ...


In [7]:
driver.quit()

### Collecting NHL Players' Facts from EliteProspects

#### Read in Players' Metadata

In [2]:
nhl_players_metadata = pd.read_csv('./data/nhl/nhl_skaters_metadata_official_ep_merge_complete_final.csv')

#### Initialize Chrome Driver for Scraping

In [36]:
from selenium.webdriver.support.wait import WebDriverWait
import undetected_chromedriver as uc

# Set up Chrome Driver ONCE
chrome_options = uc.ChromeOptions()
chrome_options.add_argument("--headless=new")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")

driver = uc.Chrome(version_main=138, options=chrome_options)
wait = WebDriverWait(driver, 15)

In [37]:
# Define output file
output_path = './data/nhl/facts/nhl_players_facts.csv'

In [38]:
def get_players_stats_by_batch(players_to_scrape):
    curr_len = len(players_to_scrape)
    fail_count = 0

    for i in range(curr_len):
        player_metadata = players_to_scrape.iloc[i]
        player_name = player_metadata['player_name']
        player_url = player_metadata['link']
        print(f"\n [{i + 1}] Collecting facts for {player_name} at {player_url}")

        try:
            player_stats = ep.get_player_facts_with_reusable_driver(player_metadata, driver, wait)

            # Write to CSV file
            if os.path.exists(output_path):
                player_stats.to_csv(output_path, mode='a', header=False, index=False, encoding='utf-8-sig')
            else:
                player_stats.to_csv(output_path, index=False, encoding='utf-8-sig')
            print(f'Successfully scraped facts for {player_name}')

            # Print Fail Rate
            print(f'Failed rate: {fail_count / (i + 1):.2f}')

            # Add random sleep to prevent getting blocked
            if i < curr_len - 1:
                sleep_time = random.uniform(10, 120)
                print(f"Sleep for {sleep_time / 60:.2f} minutes to prevent getting blocked")
                time.sleep(sleep_time)
        except Exception as e:
            print(f"Failed to get facts for {player_name}: {e}")

            fail_count += 1

            if i < curr_len - 1:
                # Sleep for 15-60 seconds before trying the next player
                sleep_time = random.uniform(10, 60)
                print(f"Sleeping for {sleep_time / 60:.2f} minutes before trying the next player")
                time.sleep(sleep_time)

### Fetch Player Facts in Batches

In [6]:
# Fetch 0 to 5 players' facts
curr_players_metadata = nhl_players_metadata[0:5]
get_players_stats_by_batch(curr_players_metadata)


 [1] Collecting facts for Jaromír Jágr at https://www.eliteprospects.com/player/8627/jaromir-jagr
Locating the main facts_list
Found 10 main facts
Extracting extra facts...
Extracted extra fact: Drafted
Extracted extra fact: Highlights
Extracting highlights...
Successfully extracted highlights.
Extracting player types...
Failed to extract player types - No player types found.
Extracting description...
Successfully extracted description.
Successfully scraped facts for Jaromír Jágr
Failed rate: 0.00
Sleep for 1.97 minutes to prevent getting blocked

 [2] Collecting facts for Joe Sakic at https://www.eliteprospects.com/player/8862/joe-sakic
Locating the main facts_list
Found 10 main facts
Extracting extra facts...
Extracted extra fact: Drafted
Extracted extra fact: Highlights
Extracting highlights...
Successfully extracted highlights.
Extracting player types...
Failed to extract player types - No player types found.
Extracting description...
Successfully extracted description.
Successful

In [7]:
# Fetch 5 to 10 players' facts
curr_players_metadata = nhl_players_metadata[5:10]
get_players_stats_by_batch(curr_players_metadata)


 [1] Collecting facts for Martin Straka at https://www.eliteprospects.com/player/8689/martin-straka
Locating the main facts_list
Found 10 main facts
Extracting extra facts...
Extracted extra fact: Drafted
Extracted extra fact: Highlights
Extracting highlights...
Successfully extracted highlights.
Extracting player types...
Failed to extract player types - No player types found.
Extracting description...
Successfully extracted description.
Successfully scraped facts for Martin Straka
Failed rate: 0.00
Sleep for 1.97 minutes to prevent getting blocked

 [2] Collecting facts for Pavel Bure at https://www.eliteprospects.com/player/16140/pavel-bure
Locating the main facts_list
Found 10 main facts
Extracting extra facts...
Extracted extra fact: Player Type
Extracted extra fact: Drafted
Extracted extra fact: Highlights
Extracting highlights...
Successfully extracted highlights.
Extracting player types...
Successfully extracted player types.
Extracting description...
Successfully extracted de

In [8]:
# Fetch 10 to 50 players' facts
curr_players_metadata = nhl_players_metadata[10:50]
get_players_stats_by_batch(curr_players_metadata)


 [1] Collecting facts for Alexei Yashin at https://www.eliteprospects.com/player/8695/alexei-yashin
Locating the main facts_list
Found 10 main facts
Extracting extra facts...
Extracted extra fact: Drafted
Extracted extra fact: Highlights
Extracting highlights...
Successfully extracted highlights.
Extracting player types...
Failed to extract player types - No player types found.
Extracting description...
Successfully extracted description.
Successfully scraped facts for Alexei Yashin
Failed rate: 0.00
Sleep for 0.78 minutes to prevent getting blocked

 [2] Collecting facts for Luc Robitaille at https://www.eliteprospects.com/player/21334/luc-robitaille
Locating the main facts_list
Found 10 main facts
Extracting extra facts...
Extracted extra fact: Drafted
Extracted extra fact: Highlights
Extracting highlights...
Successfully extracted highlights.
Extracting player types...
Failed to extract player types - No player types found.
Extracting description...
Successfully extracted descripti

In [9]:
# Fetch 50 to 100 players' facts
curr_players_metadata = nhl_players_metadata[50:100]
get_players_stats_by_batch(curr_players_metadata)


 [1] Collecting facts for Radek Dvorak at https://www.eliteprospects.com/player/8599/radek-dvorak
Locating the main facts_list
Found 10 main facts
Extracting extra facts...
Extracted extra fact: Player Type
Extracted extra fact: Drafted
Extracted extra fact: Highlights
Extracting highlights...
Successfully extracted highlights.
Extracting player types...
Successfully extracted player types.
Extracting description...
Description not found.
Successfully scraped facts for Radek Dvorak
Failed rate: 0.00
Sleep for 0.48 minutes to prevent getting blocked

 [2] Collecting facts for Mark Messier at https://www.eliteprospects.com/player/21309/mark-messier
Locating the main facts_list
Found 10 main facts
Extracting extra facts...
Extracted extra fact: Player Type
Extracted extra fact: Drafted
Extracted extra fact: Highlights
Extracting highlights...
Successfully extracted highlights.
Extracting player types...
Successfully extracted player types.
Extracting description...
Successfully extracted

In [10]:
# Fetch 100 to 200 players' facts
curr_players_metadata = nhl_players_metadata[100:200]
get_players_stats_by_batch(curr_players_metadata)


 [1] Collecting facts for Jean-Pierre Dumont at https://www.eliteprospects.com/player/8783/jean-pierre-dumont
Locating the main facts_list
Found 10 main facts
Extracting extra facts...
Extracted extra fact: Drafted
Extracted extra fact: Agency
Extracted extra fact: Highlights
Extracting highlights...
Successfully extracted highlights.
Extracting player types...
Failed to extract player types - No player types found.
Extracting description...
Description not found.
Successfully scraped facts for Jean-Pierre Dumont
Failed rate: 0.00
Sleep for 1.27 minutes to prevent getting blocked

 [2] Collecting facts for Mathieu Schneider at https://www.eliteprospects.com/player/8875/mathieu-schneider
Locating the main facts_list
Found 10 main facts
Extracting extra facts...
Extracted extra fact: Player Type
Extracted extra fact: Drafted
Extracted extra fact: Highlights
Extracting highlights...
Successfully extracted highlights.
Extracting player types...
Successfully extracted player types.
Extract

In [11]:
# Fetch 200 to 300 players' facts
curr_players_metadata = nhl_players_metadata[200:300]
get_players_stats_by_batch(curr_players_metadata)


 [1] Collecting facts for Karel Rachunek at https://www.eliteprospects.com/player/9629/karel-rachunek
Locating the main facts_list
Found 10 main facts
Extracting extra facts...
Extracted extra fact: Drafted
Extracted extra fact: Highlights
Extracting highlights...
Successfully extracted highlights.
Extracting player types...
Failed to extract player types - No player types found.
Extracting description...
Successfully extracted description.
Successfully scraped facts for Karel Rachunek
Failed rate: 0.00
Sleep for 1.22 minutes to prevent getting blocked

 [2] Collecting facts for Brad Isbister at https://www.eliteprospects.com/player/8705/brad-isbister
Locating the main facts_list
Found 10 main facts
Extracting extra facts...
Extracted extra fact: Drafted
Extracted extra fact: Highlights
Extracting highlights...
Successfully extracted highlights.
Extracting player types...
Failed to extract player types - No player types found.
Extracting description...
Description not found.
Successfu

In [12]:
# Fetch 300 to 400 players' facts
curr_players_metadata = nhl_players_metadata[300:400]
get_players_stats_by_batch(curr_players_metadata)


 [1] Collecting facts for Andrei Markov at https://www.eliteprospects.com/player/8579/andrei-markov
Locating the main facts_list
Found 10 main facts
Extracting extra facts...
Extracted extra fact: Player Type
Extracted extra fact: Drafted
Extracted extra fact: Highlights
Extracting highlights...
Successfully extracted highlights.
Extracting player types...
Successfully extracted player types.
Extracting description...
Successfully extracted description.
Successfully scraped facts for Andrei Markov
Failed rate: 0.00
Sleep for 1.15 minutes to prevent getting blocked

 [2] Collecting facts for Mike Eastwood at https://www.eliteprospects.com/player/69415/mike-eastwood
Locating the main facts_list
Found 10 main facts
Extracting extra facts...
Extracted extra fact: Drafted
Extracting highlights...
Successfully extracted highlights.
Extracting player types...
Failed to extract player types - No player types found.
Extracting description...
Successfully extracted description.
Successfully scr

In [13]:
# Fetch 400 to 500 players' facts
curr_players_metadata = nhl_players_metadata[400:500]
get_players_stats_by_batch(curr_players_metadata)


 [1] Collecting facts for Bill Houlder at https://www.eliteprospects.com/player/67298/bill-houlder
Locating the main facts_list
Found 10 main facts
Extracting extra facts...
Extracted extra fact: Drafted
Extracted extra fact: Highlights
Extracting highlights...
Successfully extracted highlights.
Extracting player types...
Failed to extract player types - No player types found.
Extracting description...
Description not found.
Successfully scraped facts for Bill Houlder
Failed rate: 0.00
Sleep for 0.24 minutes to prevent getting blocked

 [2] Collecting facts for Ossi Väänänen at https://www.eliteprospects.com/player/2685/ossi-vaananen
Locating the main facts_list
Found 10 main facts
Extracting extra facts...
Extracted extra fact: Player Type
Extracted extra fact: Drafted
Extracted extra fact: Agency
Extracted extra fact: Highlights
Extracting highlights...
Successfully extracted highlights.
Extracting player types...
Successfully extracted player types.
Extracting description...
Succes

In [14]:
# Fetch 500 to 600 players' facts
curr_players_metadata = nhl_players_metadata[500:600]
get_players_stats_by_batch(curr_players_metadata)


 [1] Collecting facts for Nathan Dempsey at https://www.eliteprospects.com/player/8741/nathan-dempsey
Locating the main facts_list
Found 10 main facts
Extracting extra facts...
Extracted extra fact: Drafted
Extracted extra fact: Highlights
Extracting highlights...
Successfully extracted highlights.
Extracting player types...
Failed to extract player types - No player types found.
Extracting description...
Description not found.
Successfully scraped facts for Nathan Dempsey
Failed rate: 0.00
Sleep for 0.19 minutes to prevent getting blocked

 [2] Collecting facts for Willie Mitchell at https://www.eliteprospects.com/player/8548/willie-mitchell
Locating the main facts_list
Found 10 main facts
Extracting extra facts...
Extracted extra fact: Player Type
Extracted extra fact: Drafted
Extracted extra fact: Highlights
Extracting highlights...
Successfully extracted highlights.
Extracting player types...
Successfully extracted player types.
Extracting description...
Description not found.
Suc

In [15]:
# Fetch 600 to 700 players' facts
curr_players_metadata = nhl_players_metadata[600:700]
get_players_stats_by_batch(curr_players_metadata)


 [1] Collecting facts for Sylvain Blouin at https://www.eliteprospects.com/player/68876/sylvain-blouin
Locating the main facts_list
Found 10 main facts
Extracting extra facts...
Extracted extra fact: Player Type
Extracted extra fact: Drafted
Extracted extra fact: Highlights
Extracting highlights...
Successfully extracted highlights.
Extracting player types...
Successfully extracted player types.
Extracting description...
Description not found.
Successfully scraped facts for Sylvain Blouin
Failed rate: 0.00
Sleep for 0.68 minutes to prevent getting blocked

 [2] Collecting facts for Blake Sloan at https://www.eliteprospects.com/player/6811/blake-sloan
Locating the main facts_list
Found 10 main facts
Extracting extra facts...
Extracted extra fact: Highlights
Extracting highlights...
Successfully extracted highlights.
Extracting player types...
Failed to extract player types - No player types found.
Extracting description...
Successfully extracted description.
Successfully scraped facts 

In [16]:
# Fetch 700 to 800 players' facts
curr_players_metadata = nhl_players_metadata[700:800]
get_players_stats_by_batch(curr_players_metadata)


 [1] Collecting facts for Mark Greig at https://www.eliteprospects.com/player/54582/mark-greig
Locating the main facts_list
Found 10 main facts
Extracting extra facts...
Extracted extra fact: Drafted
Extracted extra fact: Highlights
Extracting highlights...
Successfully extracted highlights.
Extracting player types...
Failed to extract player types - No player types found.
Extracting description...
Successfully extracted description.
Successfully scraped facts for Mark Greig
Failed rate: 0.00
Sleep for 1.40 minutes to prevent getting blocked

 [2] Collecting facts for Dmitry Afanasenkov at https://www.eliteprospects.com/player/8634/dmitry-afanasenkov
Locating the main facts_list
Found 10 main facts
Extracting extra facts...
Extracted extra fact: Drafted
Extracted extra fact: Highlights
Extracting highlights...
Successfully extracted highlights.
Extracting player types...
Failed to extract player types - No player types found.
Extracting description...
Successfully extracted descriptio

In [17]:
# Fetch 800 to 900 players' facts
curr_players_metadata = nhl_players_metadata[800:900]
get_players_stats_by_batch(curr_players_metadata)


 [1] Collecting facts for Paul Kruse at https://www.eliteprospects.com/player/27139/paul-kruse
Locating the main facts_list
Found 10 main facts
Extracting extra facts...
Extracted extra fact: Player Type
Extracted extra fact: Drafted
Extracting highlights...
Successfully extracted highlights.
Extracting player types...
Successfully extracted player types.
Extracting description...
Description not found.
Successfully scraped facts for Paul Kruse
Failed rate: 0.00
Sleep for 1.90 minutes to prevent getting blocked

 [2] Collecting facts for Sean Selmser at https://www.eliteprospects.com/player/15579/sean-selmser
Locating the main facts_list
Found 10 main facts
Extracting extra facts...
Extracted extra fact: Drafted
Extracted extra fact: Highlights
Extracting highlights...
Successfully extracted highlights.
Extracting player types...
Failed to extract player types - No player types found.
Extracting description...
Successfully extracted description.
Successfully scraped facts for Sean Sel

In [18]:
# Fetch 900 to 1000 players' facts
curr_players_metadata = nhl_players_metadata[900:1000]
get_players_stats_by_batch(curr_players_metadata)


 [1] Collecting facts for Jiri Dopita at https://www.eliteprospects.com/player/10873/jiri-dopita
Locating the main facts_list
Found 10 main facts
Extracting extra facts...
Extracted extra fact: Drafted
Extracted extra fact: Highlights
Extracting highlights...
Successfully extracted highlights.
Extracting player types...
Failed to extract player types - No player types found.
Extracting description...
Description not found.
Successfully scraped facts for Jiri Dopita
Failed rate: 0.00
Sleep for 0.93 minutes to prevent getting blocked

 [2] Collecting facts for Zdeno Cíger at https://www.eliteprospects.com/player/69982/zdeno-ciger
Locating the main facts_list
Found 10 main facts
Extracting extra facts...
Extracted extra fact: Drafted
Extracted extra fact: Highlights
Extracting highlights...
Successfully extracted highlights.
Extracting player types...
Failed to extract player types - No player types found.
Extracting description...
Successfully extracted description.
Successfully scraped

In [19]:
# Fetch 1000 to 1100 players' facts
curr_players_metadata = nhl_players_metadata[1000:1100]
get_players_stats_by_batch(curr_players_metadata)


 [1] Collecting facts for Henrik Tallinder at https://www.eliteprospects.com/player/1955/henrik-tallinder
Locating the main facts_list
Found 10 main facts
Extracting extra facts...
Extracted extra fact: Player Type
Extracted extra fact: Drafted
Extracted extra fact: Highlights
Extracting highlights...
Successfully extracted highlights.
Extracting player types...
Successfully extracted player types.
Extracting description...
Successfully extracted description.
Successfully scraped facts for Henrik Tallinder
Failed rate: 0.00
Sleep for 1.67 minutes to prevent getting blocked

 [2] Collecting facts for Brad Leeb at https://www.eliteprospects.com/player/9529/brad-leeb
Locating the main facts_list
Found 10 main facts
Extracting extra facts...
Extracted extra fact: Highlights
Extracting highlights...
Successfully extracted highlights.
Extracting player types...
Failed to extract player types - No player types found.
Extracting description...
Successfully extracted description.
Successfully 

In [20]:
# Fetch 1100 to 1200 players' facts
curr_players_metadata = nhl_players_metadata[1100:1200]
get_players_stats_by_batch(curr_players_metadata)


 [1] Collecting facts for Shawn Thornton at https://www.eliteprospects.com/player/11339/shawn-thornton
Locating the main facts_list
Found 10 main facts
Extracting extra facts...
Extracted extra fact: Player Type
Extracted extra fact: Cap Hit
Extracted extra fact: Drafted
Extracted extra fact: Agency
Extracted extra fact: Highlights
Extracting highlights...
Successfully extracted highlights.
Extracting player types...
Successfully extracted player types.
Extracting description...
Description not found.
Successfully scraped facts for Shawn Thornton
Failed rate: 0.00
Sleep for 0.51 minutes to prevent getting blocked

 [2] Collecting facts for Jason King at https://www.eliteprospects.com/player/9920/jason-king
Locating the main facts_list
Found 10 main facts
Extracting extra facts...
Extracted extra fact: Drafted
Extracted extra fact: Highlights
Extracting highlights...
Successfully extracted highlights.
Extracting player types...
Failed to extract player types - No player types found.
Ex

In [21]:
# Fetch 1200 to 1300 players' facts
curr_players_metadata = nhl_players_metadata[1200:1300]
get_players_stats_by_batch(curr_players_metadata)


 [1] Collecting facts for Fyodor Tyutin at https://www.eliteprospects.com/player/8691/fyodor-tyutin
Locating the main facts_list
Found 10 main facts
Extracting extra facts...
Extracted extra fact: Drafted
Extracted extra fact: Highlights
Extracting highlights...
Successfully extracted highlights.
Extracting player types...
Failed to extract player types - No player types found.
Extracting description...
Successfully extracted description.
Successfully scraped facts for Fyodor Tyutin
Failed rate: 0.00
Sleep for 0.99 minutes to prevent getting blocked

 [2] Collecting facts for Tim Gleason at https://www.eliteprospects.com/player/9162/tim-gleason
Locating the main facts_list
Found 10 main facts
Extracting extra facts...
Extracted extra fact: Drafted
Extracted extra fact: Highlights
Extracting highlights...
Successfully extracted highlights.
Extracting player types...
Failed to extract player types - No player types found.
Extracting description...
Successfully extracted description.
Suc

In [22]:
# Fetch 1300 to 1400 players' facts
curr_players_metadata = nhl_players_metadata[1300:1400]
get_players_stats_by_batch(curr_players_metadata)


 [1] Collecting facts for Mike Pandolfo at https://www.eliteprospects.com/player/14421/mike-pandolfo
Locating the main facts_list
Found 10 main facts
Extracting extra facts...
Extracted extra fact: Drafted
Extracted extra fact: Highlights
Extracting highlights...
Successfully extracted highlights.
Extracting player types...
Failed to extract player types - No player types found.
Extracting description...
Successfully extracted description.
Successfully scraped facts for Mike Pandolfo
Failed rate: 0.00
Sleep for 1.10 minutes to prevent getting blocked

 [2] Collecting facts for Mikhail Kuleshov at https://www.eliteprospects.com/player/39446/mikhail-kuleshov
Locating the main facts_list
Found 10 main facts
Extracting extra facts...
Extracted extra fact: Drafted
Extracted extra fact: Highlights
Extracting highlights...
Successfully extracted highlights.
Extracting player types...
Failed to extract player types - No player types found.
Extracting description...
Successfully extracted desc

In [23]:
# Fetch 1400 to 1500 players' facts
curr_players_metadata = nhl_players_metadata[1400:1500]
get_players_stats_by_batch(curr_players_metadata)


 [1] Collecting facts for Geoff Platt at https://www.eliteprospects.com/player/9469/geoff-platt
Locating the main facts_list
Found 10 main facts
Extracting extra facts...
Extracted extra fact: Highlights
Extracting highlights...
Successfully extracted highlights.
Extracting player types...
Failed to extract player types - No player types found.
Extracting description...
Description not found.
Successfully scraped facts for Geoff Platt
Failed rate: 0.00
Sleep for 1.69 minutes to prevent getting blocked

 [2] Collecting facts for Gilbert Brulé at https://www.eliteprospects.com/player/10235/gilbert-brule
Locating the main facts_list
Found 10 main facts
Extracting extra facts...
Extracted extra fact: Drafted
Extracted extra fact: Highlights
Extracting highlights...
Successfully extracted highlights.
Extracting player types...
Failed to extract player types - No player types found.
Extracting description...
Successfully extracted description.
Successfully scraped facts for Gilbert Brulé
Fa

In [24]:
# Fetch 1500 to 1600 players' facts
curr_players_metadata = nhl_players_metadata[1500:1600]
get_players_stats_by_batch(curr_players_metadata)


 [1] Collecting facts for Jean-François Jacques at https://www.eliteprospects.com/player/9026/jean-francois-jacques
Locating the main facts_list
Found 10 main facts
Extracting extra facts...
Extracted extra fact: Drafted
Extracted extra fact: Highlights
Extracting highlights...
Successfully extracted highlights.
Extracting player types...
Failed to extract player types - No player types found.
Extracting description...
Description not found.
Successfully scraped facts for Jean-François Jacques
Failed rate: 0.00
Sleep for 1.14 minutes to prevent getting blocked

 [2] Collecting facts for Dan Smith at https://www.eliteprospects.com/player/11762/dan-smith
Locating the main facts_list
Found 10 main facts
Extracting extra facts...
Extracted extra fact: Drafted
Extracted extra fact: Highlights
Extracting highlights...
Successfully extracted highlights.
Extracting player types...
Failed to extract player types - No player types found.
Extracting description...
Description not found.
Successf

In [25]:
# Scrape 1600 to 1700 players' facts
curr_players_metadata = nhl_players_metadata[1600:1700]
get_players_stats_by_batch(curr_players_metadata)


 [1] Collecting facts for Rob Schremp at https://www.eliteprospects.com/player/8876/rob-schremp
Locating the main facts_list
Found 10 main facts
Extracting extra facts...
Extracted extra fact: Player Type
Extracted extra fact: Drafted
Extracted extra fact: Highlights
Extracting highlights...
Successfully extracted highlights.
Extracting player types...
Successfully extracted player types.
Extracting description...
Successfully extracted description.
Successfully scraped facts for Rob Schremp
Failed rate: 0.00
Sleep for 1.28 minutes to prevent getting blocked

 [2] Collecting facts for Tomi Mäki at https://www.eliteprospects.com/player/3215/tomi-maki
Locating the main facts_list
Found 10 main facts
Extracting extra facts...
Extracted extra fact: Player Type
Extracted extra fact: Drafted
Extracted extra fact: Highlights
Extracting highlights...
Successfully extracted highlights.
Extracting player types...
Successfully extracted player types.
Extracting description...
Description not fou

In [26]:
# Scrape 1700 to 1800 players' facts
curr_players_metadata = nhl_players_metadata[1700:1800]
get_players_stats_by_batch(curr_players_metadata)


 [1] Collecting facts for Shawn Matthias at https://www.eliteprospects.com/player/9345/shawn-matthias
Locating the main facts_list
Found 10 main facts
Extracting extra facts...
Extracted extra fact: Drafted
Extracted extra fact: Agency
Extracted extra fact: Highlights
Extracting highlights...
Successfully extracted highlights.
Extracting player types...
Failed to extract player types - No player types found.
Extracting description...
Description not found.
Successfully scraped facts for Shawn Matthias
Failed rate: 0.00
Sleep for 0.19 minutes to prevent getting blocked

 [2] Collecting facts for Jordan Smotherman at https://www.eliteprospects.com/player/11568/jordan-smotherman
Locating the main facts_list
Found 10 main facts
Extracting extra facts...
Extracted extra fact: Drafted
Extracted extra fact: Agency
Extracted extra fact: Highlights
Extracting highlights...
Successfully extracted highlights.
Extracting player types...
Failed to extract player types - No player types found.
Extr

In [27]:
# Scrape 1800 to 1900 players' facts
curr_players_metadata = nhl_players_metadata[1800:1900]
get_players_stats_by_batch(curr_players_metadata)


 [1] Collecting facts for Luca Sbisa at https://www.eliteprospects.com/player/13946/luca-sbisa
Locating the main facts_list
Found 10 main facts
Extracting extra facts...
Extracted extra fact: Drafted
Extracted extra fact: Agency
Extracting highlights...
Successfully extracted highlights.
Extracting player types...
Failed to extract player types - No player types found.
Extracting description...
Successfully extracted description.
Successfully scraped facts for Luca Sbisa
Failed rate: 0.00
Sleep for 1.56 minutes to prevent getting blocked

 [2] Collecting facts for Jamie McGinn at https://www.eliteprospects.com/player/9336/jamie-mcginn
Locating the main facts_list
Found 10 main facts
Extracting extra facts...
Extracted extra fact: Drafted
Extracted extra fact: Agency
Extracted extra fact: Highlights
Extracting highlights...
Successfully extracted highlights.
Extracting player types...
Failed to extract player types - No player types found.
Extracting description...
Successfully extract

In [28]:
# Scrape 1900 to 2000 players' facts
curr_players_metadata = nhl_players_metadata[1900:2000]
get_players_stats_by_batch(curr_players_metadata)


 [1] Collecting facts for Andrew MacDonald at https://www.eliteprospects.com/player/12406/andrew-macdonald
Locating the main facts_list
Found 10 main facts
Extracting extra facts...
Extracted extra fact: Drafted
Extracted extra fact: Agency
Extracted extra fact: Highlights
Extracting highlights...
Successfully extracted highlights.
Extracting player types...
Failed to extract player types - No player types found.
Extracting description...
Description not found.
Successfully scraped facts for Andrew MacDonald
Failed rate: 0.00
Sleep for 1.95 minutes to prevent getting blocked

 [2] Collecting facts for Jay Beagle at https://www.eliteprospects.com/player/14825/jay-beagle
Locating the main facts_list
Found 10 main facts
Extracting extra facts...
Extracted extra fact: Player Type
Extracted extra fact: Agency
Extracted extra fact: Highlights
Extracting highlights...
Successfully extracted highlights.
Extracting player types...
Successfully extracted player types.
Extracting description...


In [29]:
# Scrape 2000 to 2100 players' facts
curr_players_metadata = nhl_players_metadata[2000:2100]
get_players_stats_by_batch(curr_players_metadata)


 [1] Collecting facts for Eric Tangradi at https://www.eliteprospects.com/player/12514/eric-tangradi
Locating the main facts_list
Found 10 main facts
Extracting extra facts...
Extracted extra fact: Drafted
Extracted extra fact: Highlights
Extracting highlights...
Successfully extracted highlights.
Extracting player types...
Failed to extract player types - No player types found.
Extracting description...
Description not found.
Successfully scraped facts for Eric Tangradi
Failed rate: 0.00
Sleep for 1.97 minutes to prevent getting blocked

 [2] Collecting facts for Maxim Noreau at https://www.eliteprospects.com/player/12153/maxim-noreau
Locating the main facts_list
Found 10 main facts
Extracting extra facts...
Extracted extra fact: Agency
Extracted extra fact: Highlights
Extracting highlights...
Successfully extracted highlights.
Extracting player types...
Failed to extract player types - No player types found.
Extracting description...
Successfully extracted description.
Successfully 

In [30]:
# Scrape 2100 to 2200 players' facts
curr_players_metadata = nhl_players_metadata[2100:2200]
get_players_stats_by_batch(curr_players_metadata)


 [1] Collecting facts for Joe Vitale at https://www.eliteprospects.com/player/15069/joe-vitale
Locating the main facts_list
Found 10 main facts
Extracting extra facts...
Extracted extra fact: Player Type
Extracted extra fact: Cap Hit
Extracted extra fact: Drafted
Extracted extra fact: Agency
Extracting highlights...
Successfully extracted highlights.
Extracting player types...
Successfully extracted player types.
Extracting description...
Successfully extracted description.
Successfully scraped facts for Joe Vitale
Failed rate: 0.00
Sleep for 0.22 minutes to prevent getting blocked

 [2] Collecting facts for Aaron Volpatti at https://www.eliteprospects.com/player/39059/aaron-volpatti
Locating the main facts_list
Found 10 main facts
Extracting extra facts...
Extracting highlights...
Successfully extracted highlights.
Extracting player types...
Failed to extract player types - No player types found.
Extracting description...
Description not found.
Successfully scraped facts for Aaron Vo

In [31]:
# Scrape 2200 to 2300 players' facts
curr_players_metadata = nhl_players_metadata[2200:2300]
get_players_stats_by_batch(curr_players_metadata)


 [1] Collecting facts for Louis Leblanc at https://www.eliteprospects.com/player/31615/louis-leblanc
Locating the main facts_list
Found 10 main facts
Extracting extra facts...
Extracted extra fact: Drafted
Extracted extra fact: Highlights
Extracting highlights...
Successfully extracted highlights.
Extracting player types...
Failed to extract player types - No player types found.
Extracting description...
Successfully extracted description.
Successfully scraped facts for Louis Leblanc
Failed rate: 0.00
Sleep for 1.01 minutes to prevent getting blocked

 [2] Collecting facts for Zack Kassian at https://www.eliteprospects.com/player/15390/zack-kassian
Locating the main facts_list
Found 10 main facts
Extracting extra facts...
Extracted extra fact: Player Type
Extracted extra fact: Drafted
Extracted extra fact: Agency
Extracted extra fact: Highlights
Extracting highlights...
Successfully extracted highlights.
Extracting player types...
Successfully extracted player types.
Extracting descri

In [32]:
# Scrape 2300 to 2400 players' facts
curr_players_metadata = nhl_players_metadata[2300:2400]
get_players_stats_by_batch(curr_players_metadata)


 [1] Collecting facts for Kevin Marshall at https://www.eliteprospects.com/player/12508/kevin-marshall
Locating the main facts_list
Found 10 main facts
Extracting extra facts...
Extracted extra fact: Drafted
Extracted extra fact: Agency
Extracted extra fact: Highlights
Extracting highlights...
Successfully extracted highlights.
Extracting player types...
Failed to extract player types - No player types found.
Extracting description...
Successfully extracted description.
Successfully scraped facts for Kevin Marshall
Failed rate: 0.00
Sleep for 0.75 minutes to prevent getting blocked

 [2] Collecting facts for Carter Ashton at https://www.eliteprospects.com/player/21033/carter-ashton
Locating the main facts_list
Found 10 main facts
Extracting extra facts...
Extracted extra fact: Player Type
Extracted extra fact: Drafted
Extracted extra fact: Agency
Extracted extra fact: Highlights
Extracting highlights...
Successfully extracted highlights.
Extracting player types...
Successfully extract

In [33]:
# Scrape 2400 to 2500 players' facts
curr_players_metadata = nhl_players_metadata[2400:2500]
get_players_stats_by_batch(curr_players_metadata)


 [1] Collecting facts for Jon Rheault at https://www.eliteprospects.com/player/14861/jon-rheault
Locating the main facts_list
Found 10 main facts
Extracting extra facts...
Extracted extra fact: Drafted
Extracted extra fact: Highlights
Extracting highlights...
Successfully extracted highlights.
Extracting player types...
Failed to extract player types - No player types found.
Extracting description...
Successfully extracted description.
Successfully scraped facts for Jon Rheault
Failed rate: 0.00
Sleep for 1.42 minutes to prevent getting blocked

 [2] Collecting facts for Derek Grant at https://www.eliteprospects.com/player/18242/derek-grant
Locating the main facts_list
Found 10 main facts
Extracting extra facts...
Extracted extra fact: Drafted
Extracted extra fact: Agency
Extracted extra fact: Highlights
Extracting highlights...
Successfully extracted highlights.
Extracting player types...
Failed to extract player types - No player types found.
Extracting description...
Description no

In [34]:
# Scrape 2500 to 2600 players' facts
curr_players_metadata = nhl_players_metadata[2500:2600]
get_players_stats_by_batch(curr_players_metadata)


 [1] Collecting facts for Petter Granberg at https://www.eliteprospects.com/player/13901/petter-granberg
Locating the main facts_list
Found 10 main facts
Extracting extra facts...
Extracted extra fact: Player Type
Extracted extra fact: Drafted
Extracted extra fact: Highlights
Extracting highlights...
Successfully extracted highlights.
Extracting player types...
Successfully extracted player types.
Extracting description...
Successfully extracted description.
Successfully scraped facts for Petter Granberg
Failed rate: 0.00
Sleep for 1.98 minutes to prevent getting blocked

 [2] Collecting facts for Julian Melchiori at https://www.eliteprospects.com/player/73778/julian-melchiori
Locating the main facts_list
Found 10 main facts
Extracting extra facts...
Extracted extra fact: Drafted
Extracted extra fact: Agency
Extracted extra fact: Highlights
Extracting highlights...
Successfully extracted highlights.
Extracting player types...
Failed to extract player types - No player types found.
Ext

In [35]:
# Scrape 2600 to 2700 players' facts
curr_players_metadata = nhl_players_metadata[2600:2700]
get_players_stats_by_batch(curr_players_metadata)


 [1] Collecting facts for Tim Schaller at https://www.eliteprospects.com/player/38059/tim-schaller
Locating the main facts_list
Found 10 main facts
Extracting extra facts...
Extracted extra fact: Agency
Extracting highlights...
Successfully extracted highlights.
Extracting player types...
Failed to extract player types - No player types found.
Extracting description...
Description not found.
Successfully scraped facts for Tim Schaller
Failed rate: 0.00
Sleep for 1.13 minutes to prevent getting blocked

 [2] Collecting facts for Petr Straka at https://www.eliteprospects.com/player/34805/petr-straka
Locating the main facts_list
Found 10 main facts
Extracting extra facts...
Extracted extra fact: Drafted
Extracted extra fact: Agency
Extracted extra fact: Highlights
Extracting highlights...
Successfully extracted highlights.
Extracting player types...
Failed to extract player types - No player types found.
Extracting description...
Successfully extracted description.
Successfully scraped f

In [36]:
# Scrape 2700 to 2800 players' facts
curr_players_metadata = nhl_players_metadata[2700:2800]
get_players_stats_by_batch(curr_players_metadata)


 [1] Collecting facts for Andreas Martinsen at https://www.eliteprospects.com/player/11958/andreas-martinsen
Locating the main facts_list
Found 10 main facts
Extracting extra facts...
Extracted extra fact: Highlights
Extracting highlights...
Successfully extracted highlights.
Extracting player types...
Failed to extract player types - No player types found.
Extracting description...
Successfully extracted description.
Successfully scraped facts for Andreas Martinsen
Failed rate: 0.00
Sleep for 1.63 minutes to prevent getting blocked

 [2] Collecting facts for Conor Sheary at https://www.eliteprospects.com/player/94484/conor-sheary
Locating the main facts_list
Found 10 main facts
Extracting extra facts...
Extracted extra fact: Agency
Extracted extra fact: Highlights
Extracting highlights...
Successfully extracted highlights.
Extracting player types...
Failed to extract player types - No player types found.
Extracting description...
Successfully extracted description.
Successfully scrap

In [37]:
# Scrape 2800 to 2900 players' facts
curr_players_metadata = nhl_players_metadata[2800:2900]
get_players_stats_by_batch(curr_players_metadata)


 [1] Collecting facts for Mike Matheson at https://www.eliteprospects.com/player/76333/mike-matheson
Locating the main facts_list
Found 10 main facts
Extracting extra facts...
Extracted extra fact: Cap Hit
Extracted extra fact: NHL Rights
Extracted extra fact: Drafted
Extracted extra fact: Agency
Extracted extra fact: Highlights
Extracting highlights...
Successfully extracted highlights.
Extracting player types...
Failed to extract player types - No player types found.
Extracting description...
Successfully extracted description.
Successfully scraped facts for Mike Matheson
Failed rate: 0.00
Sleep for 0.67 minutes to prevent getting blocked

 [2] Collecting facts for Kurtis Gabriel at https://www.eliteprospects.com/player/92683/kurtis-gabriel
Locating the main facts_list
Found 10 main facts
Extracting extra facts...
Extracted extra fact: Player Type
Extracted extra fact: Drafted
Extracted extra fact: Agency
Extracted extra fact: Highlights
Extracting highlights...
Successfully extract

In [38]:
# Scrape 2900 to 3000 players' facts
curr_players_metadata = nhl_players_metadata[2900:3000]
get_players_stats_by_batch(curr_players_metadata)


 [1] Collecting facts for Wade Megan at https://www.eliteprospects.com/player/38054/wade-megan
Locating the main facts_list
Found 10 main facts
Extracting extra facts...
Extracted extra fact: Drafted
Extracted extra fact: Agency
Extracted extra fact: Highlights
Extracting highlights...
Successfully extracted highlights.
Extracting player types...
Failed to extract player types - No player types found.
Extracting description...
Successfully extracted description.
Successfully scraped facts for Wade Megan
Failed rate: 0.00
Sleep for 0.36 minutes to prevent getting blocked

 [2] Collecting facts for Danny O'Regan at https://www.eliteprospects.com/player/121936/danny-o-regan
Locating the main facts_list
Found 10 main facts
Extracting extra facts...
Extracted extra fact: Drafted
Extracted extra fact: Agency
Extracted extra fact: Highlights
Extracting highlights...
Successfully extracted highlights.
Extracting player types...
Failed to extract player types - No player types found.
Extractin

In [39]:
# Scrape 3000 to 3100 players' facts
curr_players_metadata = nhl_players_metadata[3000:3100]
get_players_stats_by_batch(curr_players_metadata)


 [1] Collecting facts for Tanner Fritz at https://www.eliteprospects.com/player/23940/tanner-fritz
Locating the main facts_list
Found 10 main facts
Extracting extra facts...
Extracted extra fact: Agency
Extracted extra fact: Highlights
Extracting highlights...
Successfully extracted highlights.
Extracting player types...
Failed to extract player types - No player types found.
Extracting description...
Successfully extracted description.
Successfully scraped facts for Tanner Fritz
Failed rate: 0.00
Sleep for 1.45 minutes to prevent getting blocked

 [2] Collecting facts for Victor Mete at https://www.eliteprospects.com/player/232703/victor-mete
Locating the main facts_list
Found 10 main facts
Extracting extra facts...
Extracted extra fact: Drafted
Extracted extra fact: Agency
Extracted extra fact: Highlights
Extracting highlights...
Successfully extracted highlights.
Extracting player types...
Failed to extract player types - No player types found.
Extracting description...
Successfull

In [40]:
# Scrape 3100 to 3200 players' facts
curr_players_metadata = nhl_players_metadata[3100:3200]
get_players_stats_by_batch(curr_players_metadata)


 [1] Collecting facts for Mathieu Joseph at https://www.eliteprospects.com/player/227640/mathieu-joseph
Locating the main facts_list
Found 10 main facts
Extracting extra facts...
Extracted extra fact: Cap Hit
Extracted extra fact: NHL Rights
Extracted extra fact: Drafted
Extracted extra fact: Agency
Extracted extra fact: Highlights
Extracting highlights...
Successfully extracted highlights.
Extracting player types...
Failed to extract player types - No player types found.
Extracting description...
Successfully extracted description.
Successfully scraped facts for Mathieu Joseph
Failed rate: 0.00
Sleep for 1.98 minutes to prevent getting blocked

 [2] Collecting facts for Brett Howden at https://www.eliteprospects.com/player/232712/brett-howden
Locating the main facts_list
Found 10 main facts
Extracting extra facts...
Extracted extra fact: Cap Hit
Extracted extra fact: NHL Rights
Extracted extra fact: Drafted
Extracted extra fact: Agency
Extracted extra fact: Highlights
Extracting high

In [41]:
# Scrape 3200 to 3300 players' facts
curr_players_metadata = nhl_players_metadata[3200:3300]
get_players_stats_by_batch(curr_players_metadata)


 [1] Collecting facts for Riley Stillman at https://www.eliteprospects.com/player/252469/riley-stillman
Locating the main facts_list
Found 10 main facts
Extracting extra facts...
Extracted extra fact: Cap Hit
Extracted extra fact: NHL Rights
Extracted extra fact: Drafted
Extracted extra fact: Agency
Extracted extra fact: Highlights
Extracting highlights...
Successfully extracted highlights.
Extracting player types...
Failed to extract player types - No player types found.
Extracting description...
Successfully extracted description.
Successfully scraped facts for Riley Stillman
Failed rate: 0.00
Sleep for 1.73 minutes to prevent getting blocked

 [2] Collecting facts for Rem Pitlick at https://www.eliteprospects.com/player/199968/rem-pitlick
Locating the main facts_list
Found 10 main facts
Extracting extra facts...
Extracted extra fact: Drafted
Extracted extra fact: Agency
Extracted extra fact: Highlights
Extracting highlights...
Successfully extracted highlights.
Extracting player ty

In [42]:
# Scrape 3300 to 3400 players' facts
curr_players_metadata = nhl_players_metadata[3300:3400]
get_players_stats_by_batch(curr_players_metadata)


 [1] Collecting facts for Tyler Benson at https://www.eliteprospects.com/player/154220/tyler-benson
Locating the main facts_list
Found 10 main facts
Extracting extra facts...
Extracted extra fact: Player Type
Extracted extra fact: Drafted
Extracted extra fact: Highlights
Extracting highlights...
Successfully extracted highlights.
Extracting player types...
Successfully extracted player types.
Extracting description...
Successfully extracted description.
Successfully scraped facts for Tyler Benson
Failed rate: 0.00
Sleep for 0.36 minutes to prevent getting blocked

 [2] Collecting facts for Eetu Luostarinen at https://www.eliteprospects.com/player/299319/eetu-luostarinen
Locating the main facts_list
Found 10 main facts
Extracting extra facts...
Extracted extra fact: Player Type
Extracted extra fact: Cap Hit
Extracted extra fact: NHL Rights
Extracted extra fact: Drafted
Extracted extra fact: Agency
Extracted extra fact: Highlights
Extracting highlights...
Successfully extracted highligh

In [43]:
# Scrape 3400 to 3500 players' facts
curr_players_metadata = nhl_players_metadata[3400:3500]
get_players_stats_by_batch(curr_players_metadata)


 [1] Collecting facts for Joe Veleno at https://www.eliteprospects.com/player/271713/joe-veleno
Locating the main facts_list
Found 10 main facts
Extracting extra facts...
Extracted extra fact: Cap Hit
Extracted extra fact: NHL Rights
Extracted extra fact: Drafted
Extracted extra fact: Agency
Extracted extra fact: Highlights
Extracting highlights...
Successfully extracted highlights.
Extracting player types...
Failed to extract player types - No player types found.
Extracting description...
Successfully extracted description.
Successfully scraped facts for Joe Veleno
Failed rate: 0.00
Sleep for 1.79 minutes to prevent getting blocked

 [2] Collecting facts for Rasmus Kupari at https://www.eliteprospects.com/player/319079/rasmus-kupari
Locating the main facts_list
Found 10 main facts
Extracting extra facts...
Extracted extra fact: NHL Rights
Extracted extra fact: Drafted
Extracted extra fact: Agency
Extracted extra fact: Highlights
Extracting highlights...
Successfully extracted highlig

In [44]:
# Scrape 3500 to 3600 players' facts
curr_players_metadata = nhl_players_metadata[3500:3600]
get_players_stats_by_batch(curr_players_metadata)


 [1] Collecting facts for Thomas Bordeleau at https://www.eliteprospects.com/player/296636/thomas-bordeleau
Locating the main facts_list
Found 10 main facts
Extracting extra facts...
Extracted extra fact: Cap Hit
Extracted extra fact: NHL Rights
Extracted extra fact: Drafted
Extracted extra fact: Highlights
Extracting highlights...
Successfully extracted highlights.
Extracting player types...
Failed to extract player types - No player types found.
Extracting description...
Successfully extracted description.
Successfully scraped facts for Thomas Bordeleau
Failed rate: 0.00
Sleep for 1.06 minutes to prevent getting blocked

 [2] Collecting facts for Dylan Samberg at https://www.eliteprospects.com/player/201527/dylan-samberg
Locating the main facts_list
Found 10 main facts
Extracting extra facts...
Extracted extra fact: Cap Hit
Extracted extra fact: NHL Rights
Extracted extra fact: Drafted
Extracted extra fact: Agency
Extracted extra fact: Highlights
Extracting highlights...
Successfull

In [45]:
# Scrape 3600 to 3700 players' facts
curr_players_metadata = nhl_players_metadata[3600:3700]
get_players_stats_by_batch(curr_players_metadata)


 [1] Collecting facts for Chase Priskie at https://www.eliteprospects.com/player/177666/chase-priskie
Locating the main facts_list
Found 10 main facts
Extracting extra facts...
Extracted extra fact: Drafted
Extracted extra fact: Highlights
Extracting highlights...
Successfully extracted highlights.
Extracting player types...
Failed to extract player types - No player types found.
Extracting description...
Description not found.
Successfully scraped facts for Chase Priskie
Failed rate: 0.00
Sleep for 0.46 minutes to prevent getting blocked

 [2] Collecting facts for Isaak Phillips at https://www.eliteprospects.com/player/416750/isaak-phillips
Locating the main facts_list
Found 10 main facts
Extracting extra facts...
Extracted extra fact: Player Type
Extracted extra fact: Cap Hit
Extracted extra fact: NHL Rights
Extracted extra fact: Drafted
Extracting highlights...
Successfully extracted highlights.
Extracting player types...
Successfully extracted player types.
Extracting description.

In [46]:
# Scrape 3700 to 3800 players' facts
curr_players_metadata = nhl_players_metadata[3700:3800]
get_players_stats_by_batch(curr_players_metadata)


 [1] Collecting facts for Filip Král at https://www.eliteprospects.com/player/247241/filip-kral
Locating the main facts_list
Found 10 main facts
Extracting extra facts...
Extracted extra fact: Drafted
Extracted extra fact: Agency
Extracted extra fact: Highlights
Extracting highlights...
Successfully extracted highlights.
Extracting player types...
Failed to extract player types - No player types found.
Extracting description...
Description not found.
Successfully scraped facts for Filip Král
Failed rate: 0.00
Sleep for 0.33 minutes to prevent getting blocked

 [2] Collecting facts for Tristen Robins at https://www.eliteprospects.com/player/286940/tristen-robins
Locating the main facts_list
Found 10 main facts
Extracting extra facts...
Extracted extra fact: Drafted
Extracted extra fact: Agency
Extracted extra fact: Highlights
Extracting highlights...
Successfully extracted highlights.
Extracting player types...
Failed to extract player types - No player types found.
Extracting descript

In [47]:
# Scrape 3800 to len(nhl_players_metadata) players' facts
curr_players_metadata = nhl_players_metadata[3800:]
get_players_stats_by_batch(curr_players_metadata)


 [1] Collecting facts for Ondřej Pavel at https://www.eliteprospects.com/player/236950/ond%C5%99ej-pavel
Locating the main facts_list
Found 10 main facts
Extracting extra facts...
Extracted extra fact: Agency
Extracted extra fact: Highlights
Extracting highlights...
Successfully extracted highlights.
Extracting player types...
Failed to extract player types - No player types found.
Extracting description...
Description not found.
Successfully scraped facts for Ondřej Pavel
Failed rate: 0.00
Sleep for 1.60 minutes to prevent getting blocked

 [2] Collecting facts for Gage Goncalves at https://www.eliteprospects.com/player/407591/gage-goncalves
Locating the main facts_list
Found 10 main facts
Extracting extra facts...
Extracted extra fact: Cap Hit
Extracted extra fact: NHL Rights
Extracted extra fact: Drafted
Extracted extra fact: Agency
Extracted extra fact: Highlights
Extracting highlights...
Successfully extracted highlights.
Extracting player types...
Failed to extract player types 

## Check Missing Players

In [48]:
# Get unique players from nhl_players_facts.csv
if os.path.exists(output_path):
    # Read in the CSV file and set low_memory = False
    nhl_players_facts = pd.read_csv(output_path, low_memory=False)
    unique_players = set(nhl_players_facts['player_name'].unique())
else:
    unique_players = set()

In [49]:
# Find players in nhl_players_metadata not in unique_players
subset = nhl_players_metadata[0:]
players_to_scrape = subset[~subset['player_name'].isin(unique_players)].reset_index(drop=True)
players_to_scrape

,player_name,fw_def,link


In [ ]:
# Fetch Missing Players' Facts
get_players_stats_by_batch(players_to_scrape)

In [50]:
# Check the number of unique players
unique_players = set(nhl_players_facts['player_name'].unique())
len(unique_players)

3928

## Close the Driver

In [9]:
driver.quit()

### Merge player facts with links on player_name

In [92]:
# Read in nhl_players_metadata.csv
nhl_players_metadata = pd.read_csv('./data/nhl/nhl_players_metadata.csv')

In [93]:
# Read in nhl_players_facts.csv
nhl_players_facts = pd.read_csv('./data/nhl/facts/nhl_players_facts.csv')

In [94]:
# Merge the two dataframes
nhl_players_metadata_facts = pd.merge(nhl_players_metadata, nhl_players_facts, on='player_name', how='left')

In [95]:
nhl_players_metadata_facts

,player_name,fw_def,link,nation,position,height_cm,weight_kg,shoots,player_type,nhl_rights,draft,highlights,description
0,Jaromír Jágr,FW,https://www.eliteprospects.com/player/8627/jar...,Czechia,RW,189,115,L,NaN,NaN,"('1', '5', '1990')","['1-time Olympic Gold Medal', '1-time Triple G...",A true hockey legend. Very talented right wing...
1,Joe Sakic,FW,https://www.eliteprospects.com/player/8862/joe...,Canada,C,180,88,L,NaN,NaN,"('1', '15', '1987')","['1-time U20 WJC Gold Medal', '1-time World Ch...",Brother: Brian Sakic\r\nSons: Chase Sakic & Mi...
2,Patrik Elias,FW,https://www.eliteprospects.com/player/8698/pat...,Czechia / USA,LW,185,86,L,NaN,NaN,"('2', '51', '1994')","['1-time U18 EJC Bronze Medal', '2-time World ...",A forward with an excellent eye for the game a...
3,Alexei Kovalev,FW,https://www.eliteprospects.com/player/8670/ale...,Russia,RW,187,101,L,['Dangler'],NaN,"('1', '15', '1991')","['1-time Olympic Gold Medal', '1-time U20 WJC ...","Sons: Nikita Kovalev, Ivan Kovalev\r\nAlexei K..."
4,Jason Allison,FW,https://www.eliteprospects.com/player/9064/jas...,Canada,C/RW,191,98,R,NaN,NaN,"('1', '17', '1993')","['2-time U20 WJC Gold Medal', '1-time AHL Cald...",Son : Colton Allison
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3926,Nikita Grebyonkin,FW,https://www.eliteprospects.com/player/652832/n...,Russia,LW/RW,187,95,L,NaN,Philadelphia Flyers / Signed,"('5', '135', '2022')",['1-time KHL Aleksei Cherepanov (Best Rookie) ...,NaN
3927,Noah Östlund,FW,https://www.eliteprospects.com/player/535699/n...,Sweden,C/LW,181,79,L,"['Cerebral Tactician', 'Playmaker', 'Speedster']",Buffalo Sabres / Signed,"('1', '16', '2022')","['1-time J18 SM Gold Medal', '1-time U18 WJC G...","At his best, Östlund is a dynamic, shifty play..."
3928,Matt Stienburg,FW,https://www.eliteprospects.com/player/267112/m...,Canada,C,185,83,R,NaN,Colorado Avalanche / Signed,"('3', '63', '2019')",[],Father: Trevor Stienburg
3929,Nikita Prishchepov,FW,https://www.eliteprospects.com/player/691679/n...,Russia,C/LW,185,88,L,NaN,Colorado Avalanche / Signed,"('7', '217', '2024')",[],NaN


In [96]:
# Remove fw_def and rename link to player_link_ep
nhl_players_metadata_facts = nhl_players_metadata_facts.drop(['fw_def'], axis=1).rename(columns={'link': 'player_link_ep'})

In [97]:
nhl_players_metadata_facts

,player_name,player_link_ep,nation,position,height_cm,weight_kg,shoots,player_type,nhl_rights,draft,highlights,description
0,Jaromír Jágr,https://www.eliteprospects.com/player/8627/jar...,Czechia,RW,189,115,L,NaN,NaN,"('1', '5', '1990')","['1-time Olympic Gold Medal', '1-time Triple G...",A true hockey legend. Very talented right wing...
1,Joe Sakic,https://www.eliteprospects.com/player/8862/joe...,Canada,C,180,88,L,NaN,NaN,"('1', '15', '1987')","['1-time U20 WJC Gold Medal', '1-time World Ch...",Brother: Brian Sakic\r\nSons: Chase Sakic & Mi...
2,Patrik Elias,https://www.eliteprospects.com/player/8698/pat...,Czechia / USA,LW,185,86,L,NaN,NaN,"('2', '51', '1994')","['1-time U18 EJC Bronze Medal', '2-time World ...",A forward with an excellent eye for the game a...
3,Alexei Kovalev,https://www.eliteprospects.com/player/8670/ale...,Russia,RW,187,101,L,['Dangler'],NaN,"('1', '15', '1991')","['1-time Olympic Gold Medal', '1-time U20 WJC ...","Sons: Nikita Kovalev, Ivan Kovalev\r\nAlexei K..."
4,Jason Allison,https://www.eliteprospects.com/player/9064/jas...,Canada,C/RW,191,98,R,NaN,NaN,"('1', '17', '1993')","['2-time U20 WJC Gold Medal', '1-time AHL Cald...",Son : Colton Allison
...,...,...,...,...,...,...,...,...,...,...,...,...
3926,Nikita Grebyonkin,https://www.eliteprospects.com/player/652832/n...,Russia,LW/RW,187,95,L,NaN,Philadelphia Flyers / Signed,"('5', '135', '2022')",['1-time KHL Aleksei Cherepanov (Best Rookie) ...,NaN
3927,Noah Östlund,https://www.eliteprospects.com/player/535699/n...,Sweden,C/LW,181,79,L,"['Cerebral Tactician', 'Playmaker', 'Speedster']",Buffalo Sabres / Signed,"('1', '16', '2022')","['1-time J18 SM Gold Medal', '1-time U18 WJC G...","At his best, Östlund is a dynamic, shifty play..."
3928,Matt Stienburg,https://www.eliteprospects.com/player/267112/m...,Canada,C,185,83,R,NaN,Colorado Avalanche / Signed,"('3', '63', '2019')",[],Father: Trevor Stienburg
3929,Nikita Prishchepov,https://www.eliteprospects.com/player/691679/n...,Russia,C/LW,185,88,L,NaN,Colorado Avalanche / Signed,"('7', '217', '2024')",[],NaN


In [98]:
# Change all NaN to None
nhl_players_metadata_facts = nhl_players_metadata_facts.astype(object).where(pd.notnull(nhl_players_metadata_facts), None)

In [99]:
nhl_players_metadata_facts

,player_name,player_link_ep,nation,position,height_cm,weight_kg,shoots,player_type,nhl_rights,draft,highlights,description
0,Jaromír Jágr,https://www.eliteprospects.com/player/8627/jar...,Czechia,RW,189,115,L,None,None,"('1', '5', '1990')","['1-time Olympic Gold Medal', '1-time Triple G...",A true hockey legend. Very talented right wing...
1,Joe Sakic,https://www.eliteprospects.com/player/8862/joe...,Canada,C,180,88,L,None,None,"('1', '15', '1987')","['1-time U20 WJC Gold Medal', '1-time World Ch...",Brother: Brian Sakic\r\nSons: Chase Sakic & Mi...
2,Patrik Elias,https://www.eliteprospects.com/player/8698/pat...,Czechia / USA,LW,185,86,L,None,None,"('2', '51', '1994')","['1-time U18 EJC Bronze Medal', '2-time World ...",A forward with an excellent eye for the game a...
3,Alexei Kovalev,https://www.eliteprospects.com/player/8670/ale...,Russia,RW,187,101,L,['Dangler'],None,"('1', '15', '1991')","['1-time Olympic Gold Medal', '1-time U20 WJC ...","Sons: Nikita Kovalev, Ivan Kovalev\r\nAlexei K..."
4,Jason Allison,https://www.eliteprospects.com/player/9064/jas...,Canada,C/RW,191,98,R,None,None,"('1', '17', '1993')","['2-time U20 WJC Gold Medal', '1-time AHL Cald...",Son : Colton Allison
...,...,...,...,...,...,...,...,...,...,...,...,...
3926,Nikita Grebyonkin,https://www.eliteprospects.com/player/652832/n...,Russia,LW/RW,187,95,L,None,Philadelphia Flyers / Signed,"('5', '135', '2022')",['1-time KHL Aleksei Cherepanov (Best Rookie) ...,None
3927,Noah Östlund,https://www.eliteprospects.com/player/535699/n...,Sweden,C/LW,181,79,L,"['Cerebral Tactician', 'Playmaker', 'Speedster']",Buffalo Sabres / Signed,"('1', '16', '2022')","['1-time J18 SM Gold Medal', '1-time U18 WJC G...","At his best, Östlund is a dynamic, shifty play..."
3928,Matt Stienburg,https://www.eliteprospects.com/player/267112/m...,Canada,C,185,83,R,None,Colorado Avalanche / Signed,"('3', '63', '2019')",[],Father: Trevor Stienburg
3929,Nikita Prishchepov,https://www.eliteprospects.com/player/691679/n...,Russia,C/LW,185,88,L,None,Colorado Avalanche / Signed,"('7', '217', '2024')",[],None


### Write to CSV

In [100]:
# Write to CSV
nhl_players_metadata_facts.to_csv('./data/nhl/facts/nhl_players_metadata_facts_with_links.csv', index=False, encoding='utf-8-sig')


### Merge Petr' Sykora's data manullay, because of duplicate names

## Find Missing player using player_link_ep

In [142]:
# Read in nhl_players_metadata_facts_with_links.csv
nhl_players_metadata_facts = pd.read_csv('./data/nhl/facts/nhl_players_metadata_facts_with_links.csv')

In [143]:
# Read in nhl_skaters_metadata_official_ep_merge_complete_final.csv
nhl_skaters_metadata_official_ep_merge_complete_final = pd.read_csv('./data/nhl/nhl_skaters_metadata_official_ep_merge_complete_final.csv')

In [144]:
# Find missing players using player_link_ep
missing_players = nhl_skaters_metadata_official_ep_merge_complete_final[~nhl_skaters_metadata_official_ep_merge_complete_final['player_link_ep'].isin(nhl_players_metadata_facts['player_link_ep'])]

In [145]:
missing_players

,player_name,player_pos,player_link_official,player_link_ep,player_image


In [146]:
len(nhl_players_metadata_facts)

3929

In [147]:
len(nhl_skaters_metadata_official_ep_merge_complete_final)

3929

In [148]:
# Find the duplicate rows
duplicate_rows = nhl_players_metadata_facts[nhl_players_metadata_facts.duplicated(['player_name', 'player_link_ep'], keep=False)]

In [149]:
duplicate_rows

,player_name,player_link_ep,nation,position,height_cm,weight_kg,shoots,player_type,nhl_rights,draft,highlights,description


In [150]:
len(nhl_players_metadata_facts)

3929

In [ ]:
# Write to CSV